# Auto Insurance Claim Fraud Indicators and Classification

# Mary Donovan Martello

## The goal of this project was to identify significant features in fraudulent insurance claim transactions and to design predictive classification models to predict whether fraud was reported on the insurance claim transaction.  This notebook includes exploratory data analysis and data preparation for the project.

# Part 1:  Exploratory Data Analysis and Data Preparation

In [1]:
# import data set and libraries for data preparation phase
import pandas as pd
import numpy as np
# importing regex module (search strings) RegEx can be used to check if a string contains the specified search pattern
import re


### The dataset includes 1,000 prior claim transaction records.  Each record has a mix of 38 quantitative and categorical data features about the claim filed, including information on the policy, insured, and automobile, aspects of the damage incident, and elements of the claim filed.  The dataset also has a feature that indicates whether fraud was reported on each observation (i.e., either Y or N).

In [2]:
dfClaims = pd.read_csv('FradulentInsuranceClaims.csv')

In [3]:
dfClaims.head(2)

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,...,YES,71610,6510,13020,52080,Saab,92x,2004,Y,NaN
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,...,?,5070,780,780,3510,Mercedes,E400,2007,Y,NaN


### Data Cleaning

In [ ]:
# print the column names
dfClaims.columns

In [94]:
# rename difficult column names
dfClaims.rename(columns={'capital-gains': 'capital_gains', 'capital-loss': 'capital_loss'}, inplace=True)
dfClaims.columns

Index(['months_as_customer', 'age', 'policy_number', 'policy_bind_date',
       'policy_state', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'capital_gains', 'capital_loss',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_location', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', 'fraud_reported', '_c39'],
      dtype='object')

In [95]:
# check whether the data has any null values 

# There are '?' in the datset which are removed by NaN Values
dfClaims = dfClaims.replace('?',np.NaN)

dfClaims.isnull().any()



months_as_customer             False
age                            False
policy_number                  False
policy_bind_date               False
policy_state                   False
policy_csl                     False
policy_deductable              False
policy_annual_premium          False
umbrella_limit                 False
insured_zip                    False
insured_sex                    False
insured_education_level        False
insured_occupation             False
insured_hobbies                False
insured_relationship           False
capital_gains                  False
capital_loss                   False
incident_date                  False
incident_type                  False
collision_type                  True
incident_severity              False
authorities_contacted          False
incident_state                 False
incident_city                  False
incident_location              False
incident_hour_of_the_day       False
number_of_vehicles_involved    False
p

In [96]:
# check df shape
dfClaims.shape

(1000, 40)

In [97]:
# check which rows have null values
is_NaN = dfClaims.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = dfClaims[row_has_NaN]

#print(rows_with_NaN)

In [98]:
# There are '?" in three features; address these using fillna

# replace the '?' by the most common collision type 
dfClaims['collision_type'].fillna(dfClaims['collision_type'].mode()[0], inplace = True)

# property damage is either yes or no; fill with no because non responses for property damage might mean no property damage.
dfClaims['property_damage'].fillna('NO', inplace = True)

# again, if there are no responses for police report available it may mean No report available
dfClaims['police_report_available'].fillna('NO', inplace = True)

dfClaims.isnull().any().any()


True

In [99]:
# which features have null values?
dfClaims.isnull().any()

months_as_customer             False
age                            False
policy_number                  False
policy_bind_date               False
policy_state                   False
policy_csl                     False
policy_deductable              False
policy_annual_premium          False
umbrella_limit                 False
insured_zip                    False
insured_sex                    False
insured_education_level        False
insured_occupation             False
insured_hobbies                False
insured_relationship           False
capital_gains                  False
capital_loss                   False
incident_date                  False
incident_type                  False
collision_type                 False
incident_severity              False
authorities_contacted          False
incident_state                 False
incident_city                  False
incident_location              False
incident_hour_of_the_day       False
number_of_vehicles_involved    False
p

In [101]:
# removing null values to avoid errors: the _c39 feature is all null values
dfClaims.dropna(axis='columns', thresh=100, inplace = True) 

In [102]:
# confirm that all null values have been removed
dfClaims.isnull().any().any()

False

In [103]:
# confirm ?s replaced:  were 177 ?
dfClaims['collision_type'].value_counts()


Rear Collision     470
Side Collision     276
Front Collision    254
Name: collision_type, dtype: int64

In [104]:
# confirm ?s replaced:  were 360 ?
dfClaims['property_damage'].value_counts()


NO     698
YES    302
Name: property_damage, dtype: int64

In [105]:
# confirm ?s replaced:  were 342 ?
dfClaims['police_report_available'].value_counts()


NO     686
YES    314
Name: police_report_available, dtype: int64

In [106]:
# fraud counts
dfClaims.fraud_reported.value_counts().sort_index()

N    753
Y    247
Name: fraud_reported, dtype: int64

In [107]:
# split policy_csl into separate columns as it has two values per column

# new data frame with split value columns 
newDF = dfClaims["policy_csl"].str.split("/", n = 1, expand = True)

# making separate csl_bodily column from new data frame 
dfClaims["csl_bodily"] = newDF[0] 
  
# making separate csl_property column from new data frame 
dfClaims["csl_prop"] = newDF[1] 
  
# Dropping old policy_csl columns 
dfClaims.drop(columns =["policy_csl"], inplace = True) 

# convert from string to int
dfClaims["csl_bodily"] = dfClaims["csl_bodily"].astype(int)
dfClaims["csl_prop"] = dfClaims["csl_prop"].astype(int)

# df display 
dfClaims.columns

Index(['months_as_customer', 'age', 'policy_number', 'policy_bind_date',
       'policy_state', 'policy_deductable', 'policy_annual_premium',
       'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'capital_gains', 'capital_loss',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_location', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', 'fraud_reported', 'csl_bodily', 'csl_prop'],
      dtype='object')

In [108]:
# check shape again
dfClaims.shape

(1000, 40)

In [109]:
# drop umbrella outlier observation
dfClaims.drop(dfClaims[dfClaims['umbrella_limit'] < 0].index, inplace = True)
dfClaims.shape

(999, 40)

**Create derived feature for length of time claimant was a customer before claim incident.**

In [110]:
# overwriting data after changing date format 
dfClaims["incident_date"]= pd.to_datetime(dfClaims["incident_date"])
dfClaims["policy_bind_date"]= pd.to_datetime(dfClaims["policy_bind_date"])
dfClaims.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999 entries, 0 to 999
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   months_as_customer           999 non-null    int64         
 1   age                          999 non-null    int64         
 2   policy_number                999 non-null    int64         
 3   policy_bind_date             999 non-null    datetime64[ns]
 4   policy_state                 999 non-null    object        
 5   policy_deductable            999 non-null    int64         
 6   policy_annual_premium        999 non-null    float64       
 7   umbrella_limit               999 non-null    int64         
 8   insured_zip                  999 non-null    int64         
 9   insured_sex                  999 non-null    object        
 10  insured_education_level      999 non-null    object        
 11  insured_occupation           999 non-null    

In [111]:
# create new column for diff in months between policy_bind_date and incident_date
dfClaims['months_bf_incident'] = ((dfClaims.incident_date - dfClaims.policy_bind_date)/np.timedelta64(1, 'M'))
dfClaims['months_bf_incident'].astype(int)

0        3
1      102
2      173
3      295
4        8
      ... 
995    283
996     12
997    143
998     39
999    219
Name: months_bf_incident, Length: 999, dtype: int32

In [112]:
# Dropping old date coluns 
dfClaims.drop(columns =["policy_bind_date", "incident_date"], inplace = True)

In [113]:
# confirm columns
dfClaims.columns

Index(['months_as_customer', 'age', 'policy_number', 'policy_state',
       'policy_deductable', 'policy_annual_premium', 'umbrella_limit',
       'insured_zip', 'insured_sex', 'insured_education_level',
       'insured_occupation', 'insured_hobbies', 'insured_relationship',
       'capital_gains', 'capital_loss', 'incident_type', 'collision_type',
       'incident_severity', 'authorities_contacted', 'incident_state',
       'incident_city', 'incident_location', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', 'fraud_reported', 'csl_bodily', 'csl_prop',
       'months_bf_incident'],
      dtype='object')

In [114]:
# confirm shape
dfClaims.shape

(999, 39)

## Data Understanding

### This project utilized the EDA from this Exploratory Data Analysis and Hypothesis Testing project: [EDA.](https://github.com/MaryDonovanMartello/EDA-and-Hypothesis-Testing)

In [118]:
# Create new dataset with quantitative variables (exlcluding ones with too many nonunique values to be helpful)
dfClaims_quantVars = dfClaims.filter(['months_as_customer', 'age', 'policy_deductable', 'policy_annual_premium', 'umbrella_limit', 'capital_gains', 'capital_loss', 'incident_hour_of_the_day', 'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'total_claim_amount', 'injury_claim', 'property_claim', 'vehicle_claim', 'auto_year', 'csl_bodily', 'csl_prop'], axis=1)


In [119]:
dfClaims_quantVars.shape

(999, 18)

### Visuals

**This project utilized visuals from the 2_R_Visuals file and the EDA from this Exploratory Data Analysis and Hypothesis Testing project:*** [EDA.](https://github.com/MaryDonovanMartello/EDA-and-Hypothesis-Testing)

### Descriptive Statistics

In [3]:
# import libraries for statistical exploratory analysis
from collections import Counter
import random
import scipy.stats
import statsmodels.formula.api as smf
from scipy import stats
import patsy #stats
from tabulate import tabulate

**Each record in the dataset includes a label of whether the claim was classified as fraudulent (fraud = Y) or as non-fraudulent (fraud = N). Create separate subsets of the dataset so that one subset only has records that did not have fraud transactions and another subset that only has records with fraud transactions. Use these separate subsets for EDA.**

In [4]:
# subset for fraud reported if yes
rowsYesFraud = dfClaims['fraud_reported'] == 'Y'
fraudY2 = dfClaims.loc[rowsYesFraud, ]

# subset for fraud reported is no
rowsNoFraud = dfClaims['fraud_reported'] == 'N'
fraudN2 = dfClaims.loc[rowsNoFraud, ]

In [5]:
# Create new dataset with quantitative variables - full dataset
dfClaims_quant = dfClaims.filter(['months_bf_incident', 'total_claim_amount', 'witnesses', 'policy_deductable', 'umbrella_limit'], axis=1)
dfClaims_quant.describe()

,total_claim_amount,witnesses,policy_deductable,umbrella_limit
count,1000.00000,1000.000000,1000.000000,1.000000e+03
mean,52761.94000,1.487000,1136.000000,1.101000e+06
std,26401.53319,1.111335,611.864673,2.297407e+06
min,100.00000,0.000000,500.000000,-1.000000e+06
25%,41812.50000,1.000000,500.000000,0.000000e+00
50%,58055.00000,1.000000,1000.000000,0.000000e+00
75%,70592.50000,2.000000,2000.000000,0.000000e+00
max,114920.00000,3.000000,2000.000000,1.000000e+07


In [ ]:
# Create new dataset with quantitative variables - fraud reported
fraudY2_quant = fraudY2.filter(['months_bf_incident', 'total_claim_amount', 'vehicle_claim', 'witnesses', 'number_of_vehicles_involved', 'policy_deductable', 'umbrella_limit'], axis=1)
fraudY2_quant.describe()

In [ ]:
# Create new dataset with quantitative variables - no fraud reported
fraudN2_quant = fraudN2.filter(['months_bf_incident', 'total_claim_amount', 'vehicle_claim', 'witnesses', 'number_of_vehicles_involved', 'policy_deductable', 'umbrella_limit'], axis=1)
fraudN2_quant.describe()

**Correlation of quantitative features.**

In [6]:
# Create new dataset with quantitative variables - full dataframe
dfClaims_quant = dfClaims.filter(['months_bf_incident', 'total_claim_amount', 'witnesses', 'policy_deductable', 'umbrella_limit'], axis=1)
dfClaims_quant.corr()

,total_claim_amount,witnesses,policy_deductable,umbrella_limit
total_claim_amount,1.000000,-0.011114,0.022839,-0.040344
witnesses,-0.011114,1.000000,0.066639,-0.006738
policy_deductable,0.022839,0.066639,1.000000,0.010870
umbrella_limit,-0.040344,-0.006738,0.010870,1.000000


In [7]:
# Create new dataset with quantitative variables - fraud reported
fraudY2_quant = fraudY2.filter(['months_bf_incident', 'total_claim_amount', 'witnesses', 'policy_deductable', 'umbrella_limit'], axis=1)
fraudY2_quant.corr()

,total_claim_amount,witnesses,policy_deductable,umbrella_limit
total_claim_amount,1.000000,0.037589,-0.104484,-0.022993
witnesses,0.037589,1.000000,0.031163,-0.037245
policy_deductable,-0.104484,0.031163,1.000000,-0.011936
umbrella_limit,-0.022993,-0.037245,-0.011936,1.000000


In [8]:
# Create new dataset with quantitative variables - no fraud reported
fraudN2_quant = fraudN2.filter(['months_bf_incident', 'total_claim_amount', 'witnesses', 'policy_deductable', 'umbrella_limit'], axis=1)
fraudN2_quant.corr()

,total_claim_amount,witnesses,policy_deductable,umbrella_limit
total_claim_amount,1.000000,-0.032933,0.052804,-0.059233
witnesses,-0.032933,1.000000,0.077246,-0.000106
policy_deductable,0.052804,0.077246,1.000000,0.018364
umbrella_limit,-0.059233,-0.000106,0.018364,1.000000


**Point-biserial correlation is used for correlation between the binary fraud reported variable and continuous variables.  None of the point-biserial correlations for the fraud_reported feature are strong.  However, the p-value for umbrella limit shows that it may be significant.**

In [9]:
# factorize categorical from strings to numberics in dfClaims df
dfClaims['fraud_reported_f'] = pd.factorize(dfClaims.fraud_reported)[0]

In [10]:
pbcClaimAmt = stats.pointbiserialr(dfClaims.fraud_reported_f, dfClaims.total_claim_amount)
print(pbcClaimAmt)

pbcUmb = stats.pointbiserialr(dfClaims.fraud_reported_f, dfClaims.umbrella_limit)
print(pbcUmb)

pbcMonths = stats.pointbiserialr(dfClaims.fraud_reported_f, dfClaims.months_as_customer)
print(pbcMonths)

pbcWitn = stats.pointbiserialr(dfClaims.fraud_reported_f, dfClaims.witnesses)
print(pbcWitn)

pbcDeduc = stats.pointbiserialr(dfClaims.fraud_reported_f, dfClaims.policy_deductable)
print(pbcDeduc)

PointbiserialrResult(correlation=-0.16365148901480392, pvalue=1.9529359591865256e-07)
PointbiserialrResult(correlation=-0.05862165764044373, pvalue=0.06387450991600865)
PointbiserialrResult(correlation=-0.020543512494971355, pvalue=0.5164037490553202)
PointbiserialrResult(correlation=-0.04949667970707168, pvalue=0.11776506385185714)
PointbiserialrResult(correlation=-0.014817347521118537, pvalue=0.6397807865912944)


In [ ]:
# export the cleaned df to a csv file
dfClaims.to_csv('dfClaims.csv', index = False)

## Data Preparation

### Address skew in quantitative features with log-transformation.

In [120]:
# log-transformation
# first need to convert values in capital_loss to remove negative values before log_transformation.  
#     Add the largest negative number to all values.

# largest negative value
largestNeg = dfClaims['capital_loss'].min()

#dfClaims_quantVars['capital_loss'] = dfClaims_quantVars['capital_loss'].apply(lambda x: x + 1 - min(x))
dfClaims_quantVars['capital_loss'] = dfClaims_quantVars['capital_loss'] - largestNeg

In [121]:
dfClaims_quantVars['capital_loss'].min()

0

In [122]:
# log-transformation - cont

# now without negative values, can use log_transformation

def log_transformation(data):
    return data.apply(np.log1p)

dfClaims_quantVars['months_as_customer'] = log_transformation(dfClaims_quantVars['months_as_customer'])
dfClaims_quantVars['age'] = log_transformation(dfClaims_quantVars['age'])
dfClaims_quantVars['policy_deductable'] = log_transformation(dfClaims_quantVars['policy_deductable'])
dfClaims_quantVars['policy_annual_premium'] = log_transformation(dfClaims_quantVars['policy_annual_premium'])
dfClaims_quantVars['umbrella_limit'] = log_transformation(dfClaims_quantVars['umbrella_limit'])
dfClaims_quantVars['capital_gains'] = log_transformation(dfClaims_quantVars['capital_gains'])
dfClaims_quantVars['capital_loss'] = log_transformation(dfClaims_quantVars['capital_loss'])
dfClaims_quantVars['incident_hour_of_the_day'] = log_transformation(dfClaims_quantVars['incident_hour_of_the_day'])
dfClaims_quantVars['number_of_vehicles_involved'] = log_transformation(dfClaims_quantVars['number_of_vehicles_involved'])
dfClaims_quantVars['bodily_injuries'] = log_transformation(dfClaims_quantVars['bodily_injuries'])
dfClaims_quantVars['witnesses'] = log_transformation(dfClaims_quantVars['witnesses'])
dfClaims_quantVars['total_claim_amount'] = log_transformation(dfClaims_quantVars['total_claim_amount'])
dfClaims_quantVars['injury_claim'] = log_transformation(dfClaims_quantVars['injury_claim'])
dfClaims_quantVars['property_claim'] = log_transformation(dfClaims_quantVars['property_claim'])
dfClaims_quantVars['vehicle_claim'] = log_transformation(dfClaims_quantVars['vehicle_claim'])
dfClaims_quantVars['auto_year'] = log_transformation(dfClaims_quantVars['auto_year'])
dfClaims_quantVars['csl_bodily'] = log_transformation(dfClaims_quantVars['csl_bodily'])
dfClaims_quantVars['csl_prop'] = log_transformation(dfClaims_quantVars['csl_prop'])



### Scale Data

**Need to scale features for modeling purposes because of of the wide ranges of values and because PCA analysis requires scaling of features.**

In [128]:
from sklearn.preprocessing import StandardScaler
features= StandardScaler().fit_transform(dfClaims_quantVars)

### Principal Component Analysis

**Some of the variables are highly correlated to a great extent and there are still a great many features. Conduct PCA analysis for dimensionalty reduction and to address multicollinearity.**

In [48]:
#PCA
# https://www.datacamp.com/community/tutorials/principal-component-analysis-in-python
from sklearn.decomposition import PCA


In [129]:
pca=PCA(n_components=0.99, whiten=True)

In [130]:
features_pca = pca.fit_transform(features)

In [131]:
print("original number of features:", features.shape[1])

original number of features: 18


In [132]:
print("reduced number of features:", features_pca.shape[1])

reduced number of features: 16


In [133]:
# display the original feature names that compose the PCA components

model = PCA(n_components=16, whiten=True).fit(dfClaims_quantVars)
X_pc = model.transform(dfClaims_quantVars)

# number of components
n_pcs= model.components_.shape[0]

# get the index of the most important feature on EACH component
# LIST COMPREHENSION HERE
most_important = [np.abs(model.components_[i]).argmax() for i in range(n_pcs)]

initial_feature_names = ['months_as_customer', 'age', 'policy_deductable', 'policy_annual_premium', 'umbrella_limit', 'capital_gains', 'capital_loss', 'incident_hour_of_the_day', 'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'total_claim_amount', 'injury_claim', 'property_claim', 'vehicle_claim', 'auto_year', 'csl_bodily', 'csl_prop']

# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
impPca = pd.DataFrame(sorted(dic.items()))


In [134]:
print(impPca)

       0                            1
0    PC1               umbrella_limit
1   PC10                 capital_loss
2   PC11                    witnesses
3   PC12              bodily_injuries
4   PC13  number_of_vehicles_involved
5   PC14        policy_annual_premium
6   PC15                          age
7   PC16                     csl_prop
8    PC2                capital_gains
9    PC3                 injury_claim
10   PC4                 injury_claim
11   PC5           months_as_customer
12   PC6           months_as_customer
13   PC7                   csl_bodily
14   PC8     incident_hour_of_the_day
15   PC9            policy_deductable


In [136]:
# create new dataframe with PCA features
pcaDf = pd.DataFrame(data = features_pca, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 
                                                     'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16'])


In [137]:
pcaDf.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16
0,-0.483900,0.059614,-0.955847,-0.025665,0.591583,0.153248,-0.691812,0.295531,0.442051,0.051544,-1.429445,-0.710023,-0.892472,-0.512373,0.248171,0.309361
1,1.854001,-0.343416,-0.567494,-0.813326,-0.787381,0.233525,1.377591,-1.209882,-2.257126,0.971511,-0.183259,0.695840,0.060263,-0.339776,-0.659630,-0.106189
2,-0.161242,-0.969355,0.703681,1.157773,0.721673,1.432304,1.082024,-1.579674,-0.213965,-0.475423,-1.212564,-1.254666,1.275487,0.537676,-0.346111,-1.258978
3,-0.303895,0.127025,-0.483588,1.492848,0.668958,1.380413,1.505900,-0.622263,-0.473612,1.393627,-0.764645,-0.559678,-0.876623,-0.633961,1.067398,-0.204994
4,1.675210,1.074260,-1.002100,0.089537,-0.090308,0.356090,0.765507,-1.244979,-0.820860,2.178117,0.591020,-1.618367,-0.063689,0.275966,-0.648299,0.196565


In [138]:
pcaDf.shape

(999, 16)

In [139]:
# add categorical variables back to df

pcaDf['policy_state'] = dfClaims['policy_state'] 
pcaDf['insured_sex'] = dfClaims['insured_sex']
pcaDf['insured_education_level'] = dfClaims['insured_education_level']
pcaDf['insured_occupation'] = dfClaims['insured_occupation']
pcaDf['insured_hobbies'] = dfClaims['insured_hobbies']
pcaDf['insured_relationship'] = dfClaims['insured_relationship']
pcaDf['incident_type'] = dfClaims['incident_type']
pcaDf['collision_type'] = dfClaims['collision_type'] 
pcaDf['incident_severity'] = dfClaims['incident_severity']
pcaDf['authorities_contacted'] = dfClaims['authorities_contacted']
pcaDf['incident_state'] = dfClaims['incident_state']
pcaDf['incident_city'] = dfClaims['incident_city']
pcaDf['property_damage'] = dfClaims['property_damage'] 
pcaDf['police_report_available'] = dfClaims['police_report_available']
pcaDf['auto_make'] = dfClaims['auto_make']
pcaDf['auto_model'] = dfClaims['auto_model']
pcaDf['fraud_reported'] = dfClaims['fraud_reported']

In [140]:
# check for missing values in PCA dataframe
for c in pcaDf.columns:
    miss = pcaDf[c].isnull().sum()
    if miss>0:
        print("{} has {} missing value(s)".format(c,miss))

policy_state has 1 missing value(s)
insured_sex has 1 missing value(s)
insured_education_level has 1 missing value(s)
insured_occupation has 1 missing value(s)
insured_hobbies has 1 missing value(s)
insured_relationship has 1 missing value(s)
incident_type has 1 missing value(s)
collision_type has 1 missing value(s)
incident_severity has 1 missing value(s)
authorities_contacted has 1 missing value(s)
incident_state has 1 missing value(s)
incident_city has 1 missing value(s)
property_damage has 1 missing value(s)
police_report_available has 1 missing value(s)
auto_make has 1 missing value(s)
auto_model has 1 missing value(s)
fraud_reported has 1 missing value(s)


In [141]:
pcaDf.shape

(999, 33)

In [142]:
# the pcaDf now has a null row for some reason
is_NaN = pcaDf.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = pcaDf[row_has_NaN]

print(rows_with_NaN)

          PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
290 -0.275614  0.108439  0.747717  2.217323 -0.913769 -1.153043  0.527141   

          PC8       PC9      PC10  ...  collision_type  incident_severity  \
290  1.789114 -1.387921 -0.343604  ...             NaN                NaN   

     authorities_contacted  incident_state  incident_city  property_damage  \
290                    NaN             NaN            NaN              NaN   

    police_report_available auto_make auto_model fraud_reported  
290                     NaN       NaN        NaN            NaN  

[1 rows x 33 columns]


In [143]:
# remove the new null row
pcaDf =pcaDf.drop(pcaDf.index[290])

In [146]:
is_NaN = pcaDf.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = pcaDf[row_has_NaN]

print(rows_with_NaN)

Empty DataFrame
Columns: [PC1, PC2, PC3, PC4, PC5, PC6, PC7, PC8, PC9, PC10, PC11, PC12, PC13, PC14, PC15, PC16, policy_state, insured_sex, insured_education_level, insured_occupation, insured_hobbies, insured_relationship, incident_type, collision_type, incident_severity, authorities_contacted, incident_state, incident_city, property_damage, police_report_available, auto_make, auto_model, fraud_reported]
Index: []

[0 rows x 33 columns]


In [147]:
# export the revised pca df to a csv file
pcaDf.to_csv('pcaClaimsLog.csv', index = False)

> Instead of running the above code multiple times, import the dataframes for subsequent analysis.

In [38]:
# import scaled, transformed and PCA df
pcaDF = pd.read_csv('pcaClaimsLog.csv')

In [23]:
# import cleaned df without re-running the cleaning code
df = pd.read_csv('dfClaims.csv')